# 🚀 LangChain Agents Starter Notebook

This notebook provides a complete setup for getting started with LangChain Agents using Google Gemini and custom tools. It demonstrates how to create a tool-calling agent with minimal setup.

## 📋 What's Included:

1. 📦 **Dependencies Installation** - All required packages (langchain, langchain-google-genai, etc.)
2. 🔐 **Secure API Key Setup** - Support for Colab Secrets for Google API Key
3. 🧮 **Calculator Tool** - A safe mathematical expression evaluator
4. 🤖 **Agent Creation** - Complete agent setup with Google Gemini 1.5 Flash
5. 🧪 **Example Queries** - Ready-to-run test cases

## 🛠️ Usage Instructions:

1. **Set up your Google API Key:**
   - Recommended: Use the 🔑 Secrets panel on the left to add GOOGLE_API_KEY
   - Alternative: The notebook will prompt you to enter it manually if not found in Secrets.

2. **Run all cells in order** (Runtime → Run All)

3. **Customize:** Add your own tools and modify the agent prompt as needed

## 💡 Next Steps:

- Add more custom tools (web search, file operations, etc.)
- Experiment with different Gemini models
- Modify the agent prompt for specific use cases
- Build conversational interfaces

Ready to explore AI agents? Run the cells below! ⬇️

In [ ]:
# LangChain Agents Starter - Installation
# Install required dependencies
!pip install langchain langchain-openai langchain-community python-dotenv
print("✅ Dependencies installed successfully!")

In [ ]:
# Google API Key Setup (Secure)
import os
import getpass
from google.colab import userdata

# Option 1: Use Colab Secrets (Recommended)
# Go to the 🔑 icon on the left panel to add your GOOGLE_API_KEY
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("✅ Google API key loaded from Colab Secrets")
except:
    # Option 2: Manual input (less secure but works)
    print("⚠️ Google API key not found in Secrets. Please enter manually:")
    GOOGLE_API_KEY = getpass.getpass("Enter your Google API key: ")

# Set the environment variable
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
print("✅ Google API key configured successfully!")

In [ ]:
# Define Calculator Tool
from langchain.tools import tool
import operator
import ast

@tool
def calculator(expression: str) -> str:
    """
    Evaluates mathematical expressions safely.
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4")
    Returns:
        The result of the calculation as a string
    """
    try:
        # Parse the expression safely
        node = ast.parse(expression, mode='eval')
        
        # Define allowed operators
        allowed_operators = {
            ast.Add: operator.add,
            ast.Sub: operator.sub,
            ast.Mult: operator.mul,
            ast.Div: operator.truediv,
            ast.Pow: operator.pow,
            ast.Mod: operator.mod,
            ast.USub: operator.neg,
        }
        
        def eval_node(node):
            if isinstance(node, ast.Constant):
                return node.value
            elif isinstance(node, ast.BinOp):
                left = eval_node(node.left)
                right = eval_node(node.right)
                return allowed_operators[type(node.op)](left, right)
            elif isinstance(node, ast.UnaryOp):
                operand = eval_node(node.operand)
                return allowed_operators[type(node.op)](operand)
            else:
                raise ValueError(f"Unsupported operation: {type(node)}")
        
        result = eval_node(node.body)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Calculator tool defined successfully!")

In [ ]:
# Create LangChain Agent with Google Gemini and Calculator Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
import os

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,
    google_api_key=os.environ['GOOGLE_API_KEY']
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can perform calculations. When asked to calculate something, use the calculator tool to get accurate results."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Create tools list
tools = [calculator]

# Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print("✅ LangChain Agent created successfully!")
print(f"🤖 Agent has access to {len(tools)} tool(s): {[tool.name for tool in tools]}")

In [ ]:
!pip -q install "langchain[google-genai]" google-generativeai
print("✅ Gemini deps ready")

In [ ]:
# Test the Agent with Example Queries
print("✨ Testing LangChain Agent with Calculator Tool")
print("=" * 50)

# Example 1: Simple calculation
print("\n🧮 Example 1: Simple Math")
response1 = agent_executor.invoke({"input": "What is 25 * 47?"})
print(f"Question: What is 25 * 47?")
print(f"Answer: {response1['output']}")

# Example 2: Complex calculation
print("\n🧮 Example 2: Complex Math")
response2 = agent_executor.invoke({"input": "Calculate (15 + 8) * 3 - 12 / 4"})
print(f"Question: Calculate (15 + 8) * 3 - 12 / 4")
print(f"Answer: {response2['output']}")

# Example 3: Word problem
print("\n🧮 Example 3: Word Problem")
response3 = agent_executor.invoke({
    "input": "If I have 12 apples and I give away 3/4 of them, how many apples do I have left?"
})
print(f"Question: If I have 12 apples and I give away 3/4 of them, how many apples do I have left?")
print(f"Answer: {response3['output']}")

print("\n✅ All tests completed successfully!")